In [1]:
# Guides:
# https://dash.plotly.com/datatable/interactivity
# https://dash.plotly.com/datatable/style
# https://plotly.com/python/pie-charts/

# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output


# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from AnimalShelter import AnimalShelter
import base64


###########################
# Data Manipulation / Model
###########################
# Update with your username and password and CRUD Python module name. NOTE: You will
# likely need more variables for your constructor to handle the hostname and port of the MongoDB
# server, and the database and collection names

username = "aacuser"
password = "SNHU1234"
shelter = AnimalShelter(username, password)


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({})) 

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
#print(len(df.to_dict(orient='records')))
#print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Encode logo
image_filename = 'Grazioso-Salvare-Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), 
                         style={'width': '200px', 'height': '200px'})),
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard - Shawn Curtis'))),
    html.Hr(),
    html.Div([dcc.RadioItems(id = 'input-radio-button',
                              options = [dict(label = 'Water Rescue', value = 'Water'),
                                         dict(label = 'Mountain Rescue', value = 'Mountain'),
                                         dict(label = 'Disaster Rescue', value = 'Disaster'),
                                         dict(label = 'Reset', value = 'Reset')],
                              value = 'Reset'),
    html.P(id = 'output-text')]),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        filter_action="native",
        page_size= 20,
        row_selectable="single",
        selected_rows=[0],
        sort_action="native",
        sort_mode='multi',
    ),
    html.Hr(),
    
    html.Div(
        dcc.Graph(id="pie-chart"), # Pie Chart
        style={'float': 'left'}
    ),
    html.Div(
        id='map-id', # Map
        style={'float': 'right'}
    )
])

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
    
)
def update_styles(selected_columns):
    defaultCondStyle = [
    {'if': {'state': 'active'},
    'backgroundColor': '#fbff74'},
    {'if': {'state': 'selected'},
    'backgroundColor': '#00aaff'},
    {'if': {'row_index': 'odd'},
    'backgroundColor': '#c7c7c7'},
    {'if': {'row_index': 'even'},
    'backgroundColor': '#ffffff'}
]
    if selected_columns is None:
        return defaultCondStyle
    else:
        return defaultCondStyle.append({'if': {'row_index': selected_columns}, 'backgroundColor': '#fbff74'})
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can 
    # be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '800px'},
           center=[30.75,-97.48], zoom=9, children=[
           dl.TileLayer(id="base-layer-id"),
           # Marker with tool tip and popup
           # Column 13 and 14 define the grid-coordinates for 
           # the map
           # Column 4 defines the breed for the animal
           # Column 9 defines the name of the animal
           dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]],
              children=[
              dl.Tooltip(dff.iloc[row,4]),
              dl.Popup([
                 html.H1("Animal Name"),
                 html.P(dff.iloc[row,9])
             ])
          ])
       ])
    ]

# Handles logic for radio buttons and updates data table based on selection.
@app.callback(Output('datatable-id','data'),
              [Input('input-radio-button', 'value')])
def update_graph(value):
    if value == "Reset":
        # Return all records
        data = {}    
    elif value == "Water" :
        # Breed of Labrador Retriever Mix, Chesapeake Bay Retriever, or Newfoundland.
        # Prefered sex is intact female
        # Training Age is 26 weeks to 156 weeks.
        # Note: Labrador Mix includes any type of mix. That's why /Labrador is included.
        data = { "$and": [
                    {
                        "$or": [
                            {"breed": {"$regex": "Newfoundland"} }, 
                            {"breed": {"$regex": "Chesa Bay"} },
                            {"breed": {"$regex": "Labrador Retriever Mix"} },
                            {"breed": {"$regex": "/Labrador"} }
                        ],
                    },
                    {"sex_upon_outcome": "Intact Female" },
                    {"age_upon_outcome_in_weeks": { "$gte": 26, "$lte": 156 }}
                ]} 
        
    elif value == "Mountain" :
        # Breed of German Shepherd, Alaskan Malamute, Old English Sheepdog, Siberian Husky, or Rottweiler
        # Prefered sex is intact male
        # Training Age is 26 weeks to 156 weeks.
        data = { "$and": [
            {
                "$or": [
                    {"breed": "German Shepherd"}, 
                    {"breed": "Alaskan Malamute"},
                    {"breed": "Old English Sheepdog"},
                    {"breed": "Siberian Husky"},
                    {"breed": "Rottweiler"}
                ],
            },
            {"sex_upon_outcome": "Intact Male" },
            {"age_upon_outcome_in_weeks": { "$gte": 26, "$lte": 156 }}
        ]}
        
    elif value == "Disaster" :
        # Breed of Doberman Pinscher, German Shepherd, Golden Retriever, Bloodhound, or Rottweiler 
        # Prefered sex is intact male
        # Training Age is 20 weeks to 300 weeks.
        data = { "$and": [
            {
                "$or": [
                    {"breed": "Doberman Pinscher"}, 
                    {"breed": "German Shepherd"}, 
                    {"breed": "Golden Retriever"},
                    {"breed": "Bloodhound"},
                    {"breed": "Rottweiler"}
                ],
            },
            {"sex_upon_outcome": "Intact Male" },
            {"age_upon_outcome_in_weeks": { "$gte": 20, "$lte": 300 }}
        ]}
            

    df = pd.DataFrame.from_records(shelter.read(data)) 
    df.drop(columns=['_id'],inplace=True)

    data_ob = df.to_dict('records')
    return (data_ob)


@app.callback(
    Output("pie-chart", "figure"), 
    [Input("datatable-id", "data")]
)
def generate_chart(data):
    dff = pd.DataFrame.from_dict(data)
    
    fig = px.pie(
        dff,
        names='breed',
        width=1000,
        height=1000,
    )
    
    return fig

app.run_server(debug=True, port=8051)

Dash app running on http://127.0.0.1:8051/
